### Bahdanau注意力
上下文变量c在任何解码时间同步t都会被c替代，假设输入序列中有T个词元，解码时间步的上下文变量是注意力集中的输出
$$c_t=\sum_{t=1}^T\alpha(s_{t^(`)-1},h_t)h_t


In [1]:
import torch
from torch import nn
from d2l import torch as d2l

#### 定义注意力解码器
实现神经网络编码器-解码器。只需要重新定义解码器即可。为了更方便学习注意力权重，下面以AttentionDecoder类来定义带有注意力机制解码器的基本接口。

In [2]:
#@save
class AttentionDecoder(d2l.Decoder):
    """ 带有注意力机制解码器的基本接口"""
    def __init__(self, **kwargs):
        super(AttentionDecoder,self).__init__(**kwargs)

    @property
    def attention_weights(self):
        raise NotADirectoryError

让我们在接下来的Seq2SeqAttentionDeconder类中实现带有Bahdanau注意力的循环神经网络解码器。首先初始化解码器的状态，需要下面的输入：
1. 编码器在所有时间步的最终层状态，将作为注意力的键和值：
2. 上一时间步的编码器全层隐状态，将作为初始化解码器的隐状态。
3. 编码器有效长度（排除在注意力池中填充词元）
在每个解码时间步骤中，解码器上一个时间步的最终隐层状态将用作查询。因此，注意力输出和输入嵌入都连接为循环神经网络解码器的输入

In [ ]:
class Seq2SeqAttentionDeconder(AttentionDecoder):
    def __init__(self, vocab_size, embad_size, num_hiddens, num_layers, dropout=0, **kwargs):
        super(Seq2SeqAttentionDeconder, self).__init__(**kwargs)
        self.attention=d2l.AdditiveAttention(num_hiddens, num_hiddens, num_hiddens, dropout)
        self.embedding=nn.Embedding(vocab_size, embad_size)
        self.rnn=nn.GRU(embad_size+num_hiddens, num_hiddens, num_layers, dropout)
        self,dense=nn.Linear(num_hiddens, vocab_size)
    
    def init_state(self, enc_outputs, enc_valid_lens,  *args):
        # outputs的形状为（batch_size, num_steps, num_hiddens)
        # hidden_state的形状为
        return super().init_state(enc_outputs, *args)